In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from classification_models.keras import Classifiers

In [30]:
train_path = 'path to dataset of fake and real images'

In [ ]:
train_dataset = image_dataset_from_directory(
    train_path,
    image_size=(100,100),
    batch_size=32,
    label_mode='binary',
    shuffle=True,
    seed=45
)

In [33]:
# For displaying some images

for images, labels in train_dataset.take(1):
    break
plt.figure(figsize=(15, 15))
for i in range(10):
    plt.subplot(8, 8, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(labels[i].numpy())

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [35]:
# Loading the pretrained model
inceptionresnetv2, preprocess_input = Classifiers.get('inceptionresnetv2')
base_model = inceptionresnetv2(include_top = False, input_shape=(100, 100, 3), weights='imagenet')

In [36]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [39]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_dataset = train_dataset.map(process)

In [ ]:
model.fit(train_dataset, epochs=5)

In [ ]:
# Testing the model
test_path = 'path to test data'
test_dataset = image_dataset_from_directory(
    test_path,
    image_size=(100,100),
    batch_size=32,
    label_mode='binary',
    shuffle=True
)

loss, accuracy = model.evaluate(train_dataset)

In [ ]:
model.save('model/deepfake_model.h5')

In [ ]:
# Load and preprocess the image
img_path = 'PATH'  # Replace with your image path
img = image.load_img(img_path, target_size=(100, 100))  # Resize to match model input shape
img_array = image.img_to_array(img)
img_array = img_array/255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [ ]:
# Predict with the model
prediction = model.predict(img_array)

# Interpretation of the result
if prediction[0] > 0.5:
    print(f"The image is predicted to be a real with a confidence of {prediction[0][0]*100:.2f}%")
else:
    print(f"The image is predicted to be deepfake with a confidence of {(1-prediction[0][0])*100:.2f}%")

In [ ]:
# loss: 0.0620 - accuracy: 0.9709